<a href="https://colab.research.google.com/github/PabloViniciusSS/analise_rock/blob/main/analise_rock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os


df = pd.read_csv("rocksamples.csv")

In [2]:
df

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.80,88.36
1,10002,Apollo11,Soil,Unsieved,5629.00,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.00,65.56
3,10004,Apollo11,Core,Unsieved,44.80,71.76
4,10005,Apollo11,Core,Unsieved,53.40,40.31
...,...,...,...,...,...,...
2224,79528,Apollo17,Breccia,Regolith,2.38,100.00
2225,79529,Apollo17,Breccia,Regolith,1.84,100.00
2226,79535,Apollo17,Breccia,Regolith,1.69,100.00
2227,79536,Apollo17,Breccia,Regolith,1.66,100.00


In [4]:
#informações das colunas existentes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


#Dados das tabelas

 * ID - Id exclusivo para identificar a amostra
 * Missão - Identificar a missão responsavel por pegar as amostras
 * Tipo - O tipo da rocha ou outra classificação
 * Subtipo - Classificação mais especifica
 * Peso (g) - O peso original da amostra, em gramas
 * Prístina - A amostra que permanece, pois algumas são usadas em pesquisas
  

#Transformando os pesos (g) em peso(Kg)

As missões espaciais utilizam os pesos em kilogramas, mas, nos dados obtidos esses peosos das rochas estão em gramas, sendo necessário transforma-los em kg para poderem ser utilizados mais tarde.


In [5]:
df['Weight (g)'] = df['Weight (g)'].apply(lambda x : x * 0.001)
df.rename(columns={'Weight (g)': 'Weight (kg)'}, inplace=True)
df.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


In [6]:
df.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


#DataFrame de missões
Criando um Data Frame com todas as missões que tem nos dados obtidos, de forma unica para cada missão.

In [7]:
missions = pd.DataFrame()
missions['Mission'] = df['Mission'].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [8]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


#Calculando o peso total das amostras por missão

Após transformar os pesos na medida utiliza em missão e retirar todas as missões de forma unicas, agora podemos pegar esses pesos e calcular quanto de amostras cada missão teve no final.

In [10]:
sample_total_weight = df.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


#Diferença de pesos entre missões

Agora podemos identificar a diferença de extração de amostras cada uma missões, em relação uma ao outra.

In [11]:
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


In [16]:
#O total de pesos carregados em todas as missões
missions['Sample weight (kg)'].sum()

375.03598999999997

# Modulos e comandos

Para fazer uma previsão de quantas amostras as proximas missões há Lua vão poder carregar, vou adcionar as informações sobre os modulo lunares, seus pesos e comandos,no dataFrame das missões.
Os dados serão:
* Nome do módulo
* Massa do módulo
* Diferença de massa do módulo

In [17]:
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff
0,Apollo11,21.55424,NaN,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0


In [ ]:
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff
0,Apollo11,21.55424,NaN,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0


# Colocando os dados da saturno

In [ ]:
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff,Crewed area : Payload,Sample : Crewed area,Sample : Payload
0,Apollo11,21.55424,NaN,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0,0.475011,0.001043,0.000495
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0,0.483264,0.001990,0.000962
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0,0.512299,0.004149,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0,0.515310,0.004882,0.002516


Calculando a média

In [ ]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.4963026819923371
0.002848764392685611
0.0014369195019157087


# Missão Artemis

Primeiro vamos criar o DataFrame para a missão Artemis. Os dados serão:
* Missão
* Peso total (kg)
* Carga útil (kg)

In [ ]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1', 'artemis1b', 'artemis2b'],
                                'Total weight (Kg)':[artemis_crewedArea, artemis_crewedArea,artemis_crewedArea],
                                'Payload (Kg)': [26988,37965,42955]
                                })
artemis_mission

,Mission,Total weight (Kg),Payload (Kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2b,26520,42955


Vamos estimar os pesos das amostras com base na proporção das missões Apolo

In [ ]:
artemis_mission['Sample weight from total (Kg)'] = artemis_mission['Total weight (Kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (Kg)'] = artemis_mission['Payload (Kg)'] * sample_payload_ratio
artemis_mission

,Mission,Total weight (Kg),Payload (Kg),Sample weight from total (Kg),Sample weight from payload (Kg)
0,artemis1,26520,26988,75.549232,38.779584
1,artemis1b,26520,37965,75.549232,54.552649
2,artemis2b,26520,42955,75.549232,61.722877


In [ ]:
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (Kg)'] + artemis_mission['Sample weight from total (Kg)'])/2
artemis_mission

,Mission,Total weight (Kg),Payload (Kg),Sample weight from total (Kg),Sample weight from payload (Kg),Estimated sample weight (kg)
0,artemis1,26520,26988,75.549232,38.779584,57.164408
1,artemis1b,26520,37965,75.549232,54.552649,65.050940
2,artemis2b,26520,42955,75.549232,61.722877,68.636054


In [ ]:
df['Remaining (kg)'] = df['Weight (kg)'] * (df['Pristine (%)'] * .01)
df.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.0448,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.0534,40.31,0.021526


In [ ]:
df.describe()

,ID,Weight (kg),Pristine (%),Remaining (kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


In [ ]:
low_samples = df.loc[(df['Weight (kg)'] >= .16) & (df['Pristine (%)'] <= 50)]
low_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.973,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.425,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.250,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.185,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.919,35.15,0.323028


#Determinar quais rochas serão coletadas



In [ ]:
low_samples.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight (kg)     27 non-null     float64
 5   Pristine (%)    27 non-null     float64
 6   Remaining (kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


In [ ]:
low_samples.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [ ]:
df.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

In [ ]:
low_samples.groupby('Type')['Weight (kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight (kg), dtype: int64

In [ ]:
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight (kg)     22 non-null     float64
 5   Pristine (%)    22 non-null     float64
 6   Remaining (kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


In [ ]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight (kg), dtype: float64

In [ ]:
df.groupby('Type')['Weight (kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight (kg), dtype: float64

In [ ]:
needed_samples = pd.concat([needed_samples,df.loc[df['Type'] == 'Crustal']])
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              68 non-null     int64  
 1   Mission         68 non-null     object 
 2   Type            68 non-null     object 
 3   Subtype         68 non-null     object 
 4   Weight (kg)     68 non-null     float64
 5   Pristine (%)    68 non-null     float64
 6   Remaining (kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


#Relação das amostras necessárias
Sendo elas:
* Basalt
* Breccia
* Crustal

In [ ]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


#Quantidade há ser coletadas
Quantidade média de amostras de cada tipo de rocha que é preciso ser coletado


In [ ]:
needed_sample_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,4.74469


In [ ]:
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)
needed_samples_overview

,Type,Total weight (kg),Average weight (kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264813
2,Crustal,4.74469,0.103145


In [ ]:
total_rock_count = df.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of samples'].sum()
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Number of samples'] / total_rocks
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of samples,Percentage of rocks
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264813,959,0.707227
2,Crustal,4.74469,0.103145,46,0.033923


In [ ]:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

63.61713411579792

In [ ]:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks'] * artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect'] / needed_samples_overview['Average weight (kg)']
needed_samples_overview

,Type,Total weight (kg),Average weight (kg),Number of samples,Percentage of rocks,Weight to collect,Rocks to collect
0,Basalt,17.42340,1.244529,351,0.258850,16.467267,13.231731
1,Breccia,10.11850,1.264813,959,0.707227,44.991764,35.571884
2,Crustal,4.74469,0.103145,46,0.033923,2.158103,20.922917
